## キーワード検索

In [23]:
import elasticsearch, json
import pandas as pd

HOST = "192.168.1.32"
PORT = "9200"
INDEX_NAME = "accident_data"
SCROLL_TIME = '1m'
SCROLL_SIZE = 10
pd.set_option("display.max_colwidth", 100)

search_word = "カバー"

client = elasticsearch.Elasticsearch(HOST + ":" + PORT)

results = []
scroll_results = client.search(
    index = INDEX_NAME,
    scroll = SCROLL_TIME,
    size = SCROLL_SIZE,
    body = {"query": { "match": {"文章":search_word} }}
)

sid = scroll_results['_scroll_id']
scroll_results = scroll_results['hits']['hits']
scroll_size = len(scroll_results)

for scroll_result in scroll_results:
    results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['業種(大分類)'], "sentence":scroll_result['_source']['文章']})

while scroll_size > 0:
    scroll_results = client.scroll(
        scroll_id = sid,
        scroll = SCROLL_TIME )
    sid = scroll_results['_scroll_id']
    scroll_results = scroll_results['hits']['hits']
    scroll_size = len(scroll_results)
    
    for scroll_result in scroll_results:
        results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['業種(大分類)'], "sentence":scroll_result['_source']['文章']})

df = pd.DataFrame(results)
df

,id,category,sentence
0,81J8gG8BPzwrEd3sB0p6,製造業,レールを切断する鋸断機のカバーを開けて鋸断機とカバーの間に立っていたところ、鋸断機横の台車（自動運転）が動き出してカバーを押し、鋸断機とカバーの間に全身をはさまれた状態で発見されたもの。
1,h1J8gG8BPzwrEd3sB1N7,建設業,杭打ち機のメンテナンス作業中、中間振れ止めカバーが落下し、下部振れ止めカバーとの間に挟まれた。
2,5FJ8gG8BPzwrEd3sC5iP,建設業,めっき槽に付属する配管の塗装作業中、めっき槽上に取り付けてあったカバーが外されていたが、被災者はカバーがあると勘違いしてカバーの上を渡ろうとして、めっき槽の中に落ちて被災した。
3,X1J8gG8BPzwrEd3sCpGw,製造業,フェルト製品の製造作業中において、綿を送り出すための装置の下部カバー内で綿が詰まったため、被災者が詰まった綿をカバー下部から取り出していた所、綿が一度に大量に落ち、カバーから引き抜いた際にカバ...
4,z1J8gG8BPzwrEd3sCGDe,建設業,アース・オーガーで穴明け作業をするためドリルパイプの位置調整をしていたときに、土砂飛散防止用カバーを吊っていたワイヤーロープが切断したためカバーが落下して当った。
...,...,...,...
111,31J8gG8BPzwrEd3sBkYh,建設業,被災者は同僚とともに、立体駐車場内で15段目のパレット上で鉄骨などのボルトにさび止めの塗装をする作業を行っていた。その後、同僚が他の作業員を迎えに行くため昇降装置に乗り、地下1階まで下降させ、...
112,5FJ8gG8BPzwrEd3sCpGw,製造業,被災者がコンクリートパイルの骨となる鉄筋籠を製造する鉄筋籠編成機を運転したところ、スライドベースと呼ばれる溶接機をセットし忘れたのに気付き、同機械の運転を停止しないまま安全カバーを開き、回転し...
113,ClJ8gG8BPzwrEd3sBCM-,製造業,横挽き機を使って、長板から箸の長さとなる素地を切り出していた被災者が、丸ノコ盤の入ったボックス内から切りくずが吹き出たので、内部で詰まった切りくずを除去するため丸ノコのスイッチを切った。刃のボ...
114,blJ8gG8BPzwrEd3sDLtx,製造業,客先から７Ｆ（７ＦとＲＦの２ストップ）にて小荷物専用昇降機が停止しているとの故障依頼を受け現地出動したが、現地に到着しているはずの本人（被災者）と連絡が取れないため他の保守員を現地確認に向かわ...


## 類似検索したい文書を指定する

In [24]:
# 特定の行を参照する。0始まり
target_id = 10
target = df.loc[target_id]
target

id                                                                                    zVJ8gG8BPzwrEd3sCGvf
category                                                                                               建設業
sentence    自転車用道路の舗装工事中、タイヤローラーを後進させていたところ、タイヤローラー座席の台座カバー取付部が腐食していたためカバーと座席が外れ、座席ごと車外へ転落した際タイヤローラーにひかれた。
Name: 10, dtype: object

## 類似する文書を検索し、トップ10を表示する

In [26]:
# 対象の文章ベクトルを取得する。

id = target['id']
target_detail = client.search(
    index = INDEX_NAME,
    body = {"query": { "match": {"_id":id} }}
)
target_vector = target_detail['hits']['hits'][0]['_source']['scdv_vector']
target_vector

script_query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
            "source": "cosineSimilarity(params.query_vector, doc['scdv_vector']) + 1.0",
            "params": {"query_vector": target_vector}
        }
    }
}

# 類似する文書を検索する。
responses = client.search(
    index=INDEX_NAME,
    body={
        "size": SCROLL_SIZE,
        "query": script_query,
        "_source": {"includes": ["_id", "業種(大分類)", "文章"]}
    }
)
# 検索結果を整形する。1件目は検索対象の自分自身である。
responses = responses['hits']['hits']
similar = []
for response in responses:
    id = response['_id']
    score = response['_score']
    category = response['_source']['業種(大分類)']
    sentence = response['_source']['文章']
    similar.append({'ID':id, 'SCORE':score, "分類":category, "文章":sentence})

sim_pd = pd.DataFrame(similar)

sim_pd

,ID,SCORE,分類,文章
0,zVJ8gG8BPzwrEd3sCGvf,2.000000,建設業,自転車用道路の舗装工事中、タイヤローラーを後進させていたところ、タイヤローラー座席の台座カバー取付部が腐食していたためカバーと座席が外れ、座席ごと車外へ転落した際タイヤローラーにひかれた。
1,elJ8gG8BPzwrEd3sCXvJ,1.923995,建設業,道路舗装工事現場で、マカダムローラーの施工写真を撮るため、稼働中のマカダムローラーとタイヤローラーの間に入り、タイヤローラーのすぐ後でしゃがんで写真を撮っていたところ、後進してきたタイヤローラ...
2,qlJ8gG8BPzwrEd3sDcmc,1.922192,建設業,寺改装工事において、ミニクローラークレーン(1．5t)による手水鉢の設置作業を終了し、ミニクローラークレーンをトラックの荷台にバックで積載させたときに、後方に気を取られていたためクローラークレ...
3,z1J8gG8BPzwrEd3sB1d8,1.918004,建設業,新築マンション外構の駐車場舗装工事において、タイヤローラーによりアスファルトの転圧を行っていたところ、後進してきたタイヤローラーに轢かれて死亡した。
4,MFJ8gG8BPzwrEd3sDLJw,1.917308,建設業,道路改良工事において、当日の作業が終了したので、ロードローラーを運転して機械駐車場所に向ってロードローラーを後退させていたときに、運転席から転落して車体と右後輪との間に頚部を挟まれた。
5,BFJ8gG8BPzwrEd3sDLlx,1.913169,建設業,県道新設工事現場において、大型ダンプトラックから降ろされた土砂の転圧作業を行うため、タイヤローラーを運転し転圧を行っていたところ、２度目の後退時にタイヤローラーを法肩に寄せすぎたため、タイヤロ...
6,wlJ8gG8BPzwrEd3sCXfJ,1.908787,製造業,ドラグ・ショベルの修理依頼を受けた被災者が、工事現場でドラグ・ショベル後部のエンジン修理をしていたとき、工事のため路上に駐車していた4tダンプトラック（土砂をほぼ満載）が3％の傾斜路を無人で後...
7,G1J8gG8BPzwrEd3sCYTK,1.908257,運輸交通業,ダンプトラックのフロントガラスの霜取り作業を駐車場内で行っていた際、停車位置を変更しようとしてバックしてきたダンプトラックの荷台左後部と霜取りを行っていたダンプトラックの右前部の間にはさまれた。
8,nlJ8gG8BPzwrEd3sDcac,1.902783,建設業,看板の設置に伴う基礎工事のため、掘削準備のためバックホーをバックで移動中に、ダンプトラックの誘導を終えて歩行中の者をクローラーベルトでひいた。
9,sVJ8gG8BPzwrEd3sBDJG,1.901540,建設業,駐車場の舗装工事において、舗装区画外にこぼれたアスファルト合材をダンプトラックに積み込む作業中に、タイヤローラーが減速しながら後退してきたので退避しようとしたときにタイヤローラーが急加速したた...
